<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/Alpha%E2%80%93Omega_Engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Alpha–Omega Engine: iterative scaffold with typed interfaces, JSON-safe checkpoints,
and signed omega imprints. Includes a minimal NumPy-based demo domain.
"""

from __future__ import annotations

import base64
import hmac
import hashlib
import json
import time
import uuid
from dataclasses import dataclass, asdict
from typing import Any, Callable, Dict, List, NamedTuple, Optional, Tuple

# Optional: comment out NumPy imports if you will plug in a non-NumPy domain.
import numpy as np


# -----------------------------------------------------------------------------
# Domain types
# -----------------------------------------------------------------------------

@dataclass(frozen=True)
class State:
    payload: Any
    alpha: int
    cycle: int
    seed: int
    meta: Dict[str, Any]


@dataclass
class Metrics:
    similarity: float  # [0,1], higher is closer to Ω
    energy: float      # domain-specific cost to minimize
    changed: bool      # did state meaningfully change this step?

    def to_dict(self) -> Dict[str, Any]:
        return {"similarity": self.similarity, "energy": self.energy, "changed": self.changed}


@dataclass(frozen=True)
class OmegaTarget:
    signature: Any     # Define what Ω means in your domain (vector, template, distribution, etc.)
    threshold: float   # convergence threshold on your chosen metric


@dataclass
class Config:
    max_cycles: int = 10_000
    planck_step: float = 1.0
    alpha_increment_on_omega: int = 1
    stagnation_window: int = 50
    min_delta: float = 1e-6
    checkpoint_every: int = 100
    hard_timeout_s: Optional[float] = None
    payload_serialization: str = "list"  # "list" | "base64"
    schema_version: str = "1.0.0"


class TraceEvent(NamedTuple):
    cycle: int
    alpha: int
    metrics: Metrics
    note: str


@dataclass
class Result:
    status: str                 # "stasis", "timeout", "max_cycles"
    final_state: State
    trace: List[TraceEvent]
    omega_hits: int


# -----------------------------------------------------------------------------
# Plug-in interfaces
# -----------------------------------------------------------------------------

SeedFn = Callable[[int, Dict[str, Any]], State]
EvolveFn = Callable[[State, float], Any]
CollapseFn = Callable[[Any], Any]
RecurseFn = Callable[[Any, State], State]  # (collapsed, prev_state) -> next_state
MeasureFn = Callable[[State, OmegaTarget], Metrics]
CheckpointFn = Callable[[State, Metrics, str, Config], None]
OmegaStoreFn = Callable[[State, Metrics, str, Config], None]


# -----------------------------------------------------------------------------
# Serialization helpers (NumPy-safe)
# -----------------------------------------------------------------------------

def payload_to_jsonable(payload: Any, mode: str = "list") -> Any:
    """
    Convert common payload types into JSON-serializable forms.
    - mode="list": np.ndarray -> nested lists (human-readable, diffable)
    - mode="base64": np.ndarray -> dict with base64 bytes, shape, dtype (compact/precise)
    Fallback: return payload if already JSON-serializable.
    """
    if isinstance(payload, np.ndarray):
        if mode == "base64":
            return {
                "encoding": "base64-ndarray",
                "shape": list(payload.shape),
                "dtype": str(payload.dtype),
                "data": base64.b64encode(payload.tobytes()).decode("ascii"),
            }
        # default to list
        return payload.tolist()
    # Add more custom cases here if your domain uses other non-JSON types.
    return payload


def json_dumps_canonical(obj: Any) -> str:
    """
    Canonical JSON string for signing (stable key order, compact).
    """
    return json.dumps(obj, sort_keys=True, separators=(",", ":"))


# -----------------------------------------------------------------------------
# Default checkpoint and omega storage (files, HMAC-signed omega)
# -----------------------------------------------------------------------------

def checkpoint_file(state: State, metrics: Metrics, run_id: str, cfg: Config) -> None:
    blob = {
        "schema_version": cfg.schema_version,
        "run_id": run_id,
        "cycle": state.cycle,
        "alpha": state.alpha,
        "seed": state.seed,
        "meta": state.meta,
        "metrics": metrics.to_dict(),
        "payload": payload_to_jsonable(state.payload, mode=cfg.payload_serialization),
    }
    path = f"chkpt_{run_id}_{state.cycle}.json"
    with open(path, "w", encoding="utf-8") as f:
        json.dump(blob, f, ensure_ascii=False, indent=2, sort_keys=True)


def store_omega_signed(
    state: State,
    metrics: Metrics,
    run_id: str,
    cfg: Config,
    key: bytes = b"replace_me_with_a_secret_key",
) -> None:
    data = {
        "schema_version": cfg.schema_version,
        "run_id": run_id,
        "cycle": state.cycle,
        "alpha": state.alpha,
        "seed": state.seed,
        "meta": state.meta,
        "metrics": metrics.to_dict(),
        "payload": payload_to_jsonable(state.payload, mode=cfg.payload_serialization),
    }
    payload_str = json_dumps_canonical(data).encode("utf-8")
    sig = hmac.new(key, payload_str, hashlib.sha256).hexdigest()
    out = {"data": data, "signature": {"alg": "HMAC-SHA256", "value": sig}}
    path = f"omega_{run_id}_{state.cycle}.json"
    with open(path, "w", encoding="utf-8") as f:
        json.dump(out, f, ensure_ascii=False, indent=2, sort_keys=True)


# -----------------------------------------------------------------------------
# Engine
# -----------------------------------------------------------------------------

def alpha_omega_engine(
    seed_fn: SeedFn,
    evolve_field: EvolveFn,
    collapse: CollapseFn,
    recurse: RecurseFn,
    measure: MeasureFn,
    checkpoint: Optional[CheckpointFn],
    store_omega: Optional[OmegaStoreFn],
    *,
    seed: int,
    omega: OmegaTarget,
    cfg: Config,
    init_meta: Optional[Dict[str, Any]] = None,
) -> Result:
    run_id = f"AOE-{uuid.uuid4().hex[:8]}"
    start = time.time()
    meta = {"run_id": run_id, "schema_version": cfg.schema_version, **(init_meta or {})}

    state = seed_fn(seed, meta)
    trace: List[TraceEvent] = []
    omega_hits = 0
    stagnation: List[float] = []
    prev_payload = state.payload

    for cycle in range(cfg.max_cycles):
        if cfg.hard_timeout_s and time.time() - start > cfg.hard_timeout_s:
            return Result("timeout", state, trace, omega_hits)

        # 1. Field projection
        field = evolve_field(state, cfg.planck_step)

        # 2. Observer-feedback collapse
        collapsed = collapse(field)

        # 3. Harmonic recursion (domain transform to a new State)
        next_state = recurse(collapsed, state)

        # 4. Measurement against Ω
        metrics = measure(next_state, omega)

        # 5. Changed detection vs previous payload (NumPy-aware)
        changed = True
        try:
            if isinstance(prev_payload, np.ndarray) and isinstance(next_state.payload, np.ndarray):
                changed = bool(np.any(np.not_equal(prev_payload, next_state.payload)))
            else:
                changed = (prev_payload != next_state.payload)
        except Exception:
            changed = True  # be conservative if equality fails
        metrics.changed = changed

        # 6. Stagnation tracking
        stagnation.append(metrics.similarity)
        if len(stagnation) > cfg.stagnation_window:
            stagnation.pop(0)

        # 7. Checkpointing
        if checkpoint and cfg.checkpoint_every > 0 and (cycle % cfg.checkpoint_every == 0):
            checkpoint(next_state, metrics, run_id, cfg)

        # 8. Omega convergence (hysteresis could be added externally if needed)
        if metrics.similarity >= omega.threshold:
            omega_hits += 1
            if store_omega:
                store_omega(next_state, metrics, run_id, cfg)
            # α progression as curriculum, not a reset
            next_state = State(
                payload=next_state.payload,
                alpha=next_state.alpha + cfg.alpha_increment_on_omega,
                cycle=next_state.cycle,
                seed=next_state.seed,
                meta=next_state.meta,
            )

        # 9. Ω-Stasis detection (flat similarity and no meaningful change)
        if (not metrics.changed) and len(stagnation) == cfg.stagnation_window:
            window_span = max(stagnation) - min(stagnation)
            if window_span < cfg.min_delta:
                trace.append(TraceEvent(next_state.cycle, next_state.alpha, metrics, "Ω-Stasis"))
                return Result("stasis", next_state, trace, omega_hits)

        # 10. Advance
        prev_payload = state.payload
        state = State(
            payload=next_state.payload,
            alpha=next_state.alpha,
            cycle=cycle + 1,
            seed=state.seed,
            meta=next_state.meta,
        )
        trace.append(TraceEvent(state.cycle, state.alpha, metrics, "step"))

    return Result("max_cycles", state, trace, omega_hits)


# -----------------------------------------------------------------------------
# Minimal NumPy vector domain (demo)
# -----------------------------------------------------------------------------

def seed_fn(seed: int, meta: Dict[str, Any]) -> State:
    rng = np.random.default_rng(seed)
    vec = rng.normal(size=128).astype(np.float32)
    return State(payload=vec, alpha=0, cycle=0, seed=seed, meta=meta)


def evolve_field(state: State, step: float) -> np.ndarray:
    # Simple linear drift with α-scaled contraction toward origin
    vec = state.payload
    assert isinstance(vec, np.ndarray), "Vector domain expects np.ndarray payload"
    drift = -0.01 * vec + 0.001 * state.alpha
    return vec + step * drift


def collapse(field: np.ndarray) -> np.ndarray:
    # Identity collapse for demo; plug in measurement/selection as needed
    return field


def recurse(collapsed: np.ndarray, prev_state: State) -> State:
    # Carry forward meta/seed; alpha progression handled by engine on Ω
    return State(
        payload=collapsed,
        alpha=prev_state.alpha,
        cycle=prev_state.cycle + 1,
        seed=prev_state.seed,
        meta=prev_state.meta,
    )


def measure(state: State, omega: OmegaTarget) -> Metrics:
    x = state.payload
    o = omega.signature
    assert isinstance(x, np.ndarray) and isinstance(o, np.ndarray), "Vector domain measure expects arrays"
    denom = float(np.linalg.norm(x) * np.linalg.norm(o) + 1e-12)
    cos_sim = float(np.dot(x, o) / denom)
    sim_01 = (cos_sim + 1.0) / 2.0  # normalize to [0,1]
    energy = float(np.linalg.norm(x - o))
    return Metrics(similarity=sim_01, energy=energy, changed=True)  # 'changed' set by engine


# -----------------------------------------------------------------------------
# Run demo
# -----------------------------------------------------------------------------

def main() -> None:
    # Define Ω as the all-ones vector for demo
    omega_vec = np.ones(128, dtype=np.float32)
    omega = OmegaTarget(signature=omega_vec, threshold=0.95)

    cfg = Config(
        max_cycles=2000,
        checkpoint_every=200,
        stagnation_window=20,
        min_delta=1e-5,
        planck_step=1.0,
        payload_serialization="list",  # switch to "base64" for compact/precise artifacts
        hard_timeout_s=10.0,
    )

    res = alpha_omega_engine(
        seed_fn=seed_fn,
        evolve_field=evolve_field,
        collapse=collapse,
        recurse=recurse,
        measure=measure,
        checkpoint=checkpoint_file,
        store_omega=lambda s, m, r, c: store_omega_signed(s, m, r, c, key=b"demo_key_do_not_use_in_prod"),
        seed=42,
        omega=omega,
        cfg=cfg,
        init_meta={"experiment": "demo-vector"},
    )

    print(f"Status: {res.status}")
    print(f"Cycles: {res.final_state.cycle}")
    print(f"Alpha: {res.final_state.alpha}")
    print(f"Omega hits: {res.omega_hits}")
    if res.trace:
        last = res.trace[-1]
        print(f"Final similarity: {last.metrics.similarity:.6f}, energy: {last.metrics.energy:.6f}, changed: {last.metrics.changed}")


if __name__ == "__main__":
    main()